# Problem 194: Coloured Configurations

Consider graphs built with the units $A$: 

![](./0194_GraphA.png)

and $B$: 

![](./0194_GraphB.png)

where the units are glued along the vertical edges as in the graph 

![](./0194_Fig.png)

A configuration of type $(a, b, c)$ is a graph thus built of $a$ units $A$ and $b$ units $B$, where the graph's vertices are coloured using up to $c$ colours, so that no two adjacent vertices have the same colour.
The compound graph above is an example of a configuration of type $(2,2,6)$, in fact of type $(2,2,c)$ for all $c \ge 4$.

Let $N(a, b, c)$ be the number of configurations of type $(a, b, c)$.
For example, $N(1,0,3) = 24$, $N(0,2,4) = 92928$ and $N(2,2,3) = 20736$.

Find the last $8$ digits of $N(25,75,1984)$.

In [89]:
graphA = {
    0: [1, 2, 5],
    1: [0, 2, 6],
    2: [0, 1, 3],
    3: [2, 4],
    4: [3, 5, 6],
    5: [0, 4, 6],
    6: [1, 4, 5]
}

graphB = {
    0: [2, 5],
    1: [2, 6],
    2: [0, 1, 3],
    3: [2, 4],
    4: [3, 5, 6],
    5: [0, 4, 6],
    6: [1, 4, 5]
}

completeGraph = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1]
}

def countColouredConfigurations(graph: dict[list[int]], numColors=int) -> int:
    colors = [x for x in range(numColors)]
    count = [0]

    def backtrack(vertex: int, currentColors: list[int]) -> int:
        if vertex == 0: # first pass
            currentColors[0] = colors[0]
            backtrack(1, currentColors.copy())
        elif vertex >= len(graph): # bottom of spanning tree
            count[0] = count[0] + 1 if currentColors[-1] >= 0 else count[0]
            return
        else: 
            if currentColors[vertex-1] < 0: # no valid colorings
                return
            for color in colors:
                if color not in [currentColors[neighbour] for neighbour in graph[vertex]]:
                    currentColors[vertex] = color
                    backtrack(vertex + 1, currentColors.copy())
    
    currentColors = [-1 for x in range(len(graph))]
    backtrack(0, currentColors)
    return numColors*count[0]

In [96]:
countColouredConfigurations(graphB, 10)

3824640

In [4]:
0 in [1, 0]

True